# scqubits example: composite Hilbert spaces
J. Koch and P. Groszkowski<br>
E. Blackwell

For further documentation of scqubits see https://scqubits.readthedocs.io/en/latest/.

---

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import scqubits as scq

import numpy as np
import qutip as qt


# Working with composite Hilbert spaces and interfacing with QuTiP

Systems of interest for quantum information processing will involve multiple qubits as well as oscillators with mutual coupling. The resulting Hilbert space is the tensor product of the individual constituent Hilbert spaces. The `HilbertSpace` class allows one to define such coupled systems, to define the interactions between them, and to contruct the overall Hamiltonian. From this, dressed eigenenergies and eigenstates can be extracted. The operator matrices and state vectors  at the `HilbertSpace` level are given as QuTiP `Qobj` instances. This interface to QuTiP is particularly helpful if the task at hand is the simulation of time-dependent dynamics of the coupled system, perhaps including additional drive terms.

## Example: two ocilllator qubits coupled by an rf SQUID

As an interesting example of a coupled quantum system consider two harmonic modes coupled by an rf SQUID. The effective Hamiltonian describing this composite system, after integrating out the SQUID degrees of freedom,  is given by [https://www.nature.com/articles/s41567-019-0703-5]

$\displaystyle H/\hbar = \omega_a a^\dagger a + \omega_b b^\dagger b + g(a^\dagger b + a b^\dagger) + g_2(a^\dagger a^\dagger b + a a b^\dagger) + \chi_{ab} a^\dagger ab^\dagger b + \frac{\chi_{aa}}{2} a^\dagger a^\dagger a a + \frac{\chi_{bb}}{2} b^\dagger b^\dagger b b$.


### Define Hilbert space components, initialize `HilbertSpace` object

To set up the Hilbert space, we define the separate components as Kerr oscillators and initialize a `HilbertSpace` object by submitting the list of all subsystems:

In [2]:
# Set up the components / subspaces of our Hilbert space

osc1 = scq.KerrOscillator(
    E_osc = 4.284,
    K = 0.003,
    l_osc = 1,
    truncated_dim = 4,
)

osc2 = scq.KerrOscillator(
    E_osc = 7.073,
    K = 0.015,
    l_osc = 1,
    truncated_dim = 4,
)

# Form a list of all components making up the Hilbert space.
hilbertspace = scq.HilbertSpace([osc1, osc2])

In [3]:
print(hilbertspace)

HilbertSpace:  subsystems
-------------------------

KerrOscillator------| [KerrOscillator_1]
                    | E_osc: 4.284
                    | K: 0.003
                    | l_osc: 1
                    | truncated_dim: 4
                    |
                    | dim: 4


KerrOscillator------| [KerrOscillator_2]
                    | E_osc: 7.073
                    | K: 0.015
                    | l_osc: 1
                    | truncated_dim: 4
                    |
                    | dim: 4




While we yet have to set up the interactions between the components, we can already obtain the bare Hamiltonian of the non-interacting subsystems, expressed as a matrix in the joint Hilbert space:

In [4]:
bare_hamiltonian = hilbertspace.bare_hamiltonian()
bare_hamiltonian

Quantum object: dims = [[4, 4], [4, 4]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[ 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     7.073  0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.    14.116  0.     0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.    21.129  0.     0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     4.284  0.     0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.    11.357  0.     0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.    18.4    0.     0.     0.
   0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.    25.413  0.     0.
   0.     0.     0.     0.     0.  

The Hamiltonian matrix is given in the form of a QuTiP quantum object.
This facilitates simple incorporation of Hamiltonians generated with `scqubits` into any of the solvers QuTiP offers for time evolution.

### Set up interaction terms between individual subsystems

1. $g(a^\dagger b + a b^\dagger)$
2. $g_2(a^\dagger a^\dagger b + a a b^\dagger)$
3. $\chi_{ab} a^\dagger ab^\dagger b$

In [5]:
# Term 1
# Option A: operator product specification via callables

g = 0.1

hilbertspace.add_interaction(
    g_strength = g,
    op1 = osc1.creation_operator,
    op2 = osc2.annihilation_operator,
    add_hc = True
)


# Term 2
# Option B: string-based specification of interaction term

g2 = 0.035 

hilbertspace.add_interaction(
    expr = "g2 * (adag * adag * b)",
    op1 = ("adag", osc1.creation_operator),
    op2 = ("b", osc2.annihilation_operator),
    add_hc = True
)


# Term 3

chi_ab = 0.01

hilbertspace.add_interaction(
    expr = "chi_ab * a.dag() * a.dag() * b.dag() * b",
    op1 = ("a", osc1.annihilation_operator),
    op2 = ("b", osc2.annihilation_operator),
    add_hc = True
)



Now that the interactions are specified, the full Hamiltonian of the coupled system can be obtained via:

In [6]:
dressed_hamiltonian = hilbertspace.hamiltonian()
dressed_hamiltonian

Quantum object: dims = [[4, 4], [4, 4]], shape = (16, 16), type = oper, isherm = True
Qobj data =
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 7.07300000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.94974747e-02 1.41421356e-02 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.41160000e+01 0.00000000e+00
  0.00000000e+00 1.41421356e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 7.00000000e-02 2.82842712e-02 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.11290000e+01
  0.00000000e+00 0.00000000e+00 1.73205081e-01 0.00000000e+00
  0.00000000e+00 0.00000000e+00

Since the composite Hamiltonian is a `qutip.Qobj`, The eigenvalues and eigenvectors can be now be obtained via the usual QuTiP routine:

In [7]:
evals, evecs = dressed_hamiltonian.eigenstates(eigvals=4)
print(evals)

[0.         4.28041833 7.07490698 8.55649874]


### GUI use for `HilbertSpace` object creation

As an alternative to the programmatic generation of a new `HilbertSpace` object, the following GUI-based creation process is supported:

In [8]:
hilbertspace_new = scq.HilbertSpace.create()

Output()

### Spectrum lookup and converting between bare and dressed indices

To use lookup functions for state indices, energies and states, first generate the lookup table via:

In [9]:
hilbertspace.generate_lookup()

Here are the bare energies of the first Kerr oscillator:

In [10]:
hilbertspace.lookup.bare_eigenenergies(osc1)

array([ 0.   ,  4.284,  8.562, 12.834])

The dressed state with index j=8 corresponds to following bare product state:

In [11]:
hilbertspace.lookup.bare_index(8)

(1, 2)

And the bare product state (2,1) most closely matches the following dressed state:

In [12]:
hilbertspace.lookup.dressed_index((2,1))

7

This is the eigenenergy for dressed index j=10:

In [13]:
hilbertspace.lookup.energy_dressed_index(10)

21.135009003451163

## Sweeping over an external parameter

scqubits provides the class `ParameterSweep` to facilitate computation of spectra as function of an external parameter. See the example notebook for `ParameterSweep` to explore sweeps and visualizing sweep data.